In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [ ]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# ✅ Load Ganga LLM
model_name = "LingoIITGN/ganga-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset
import random

# ✅ Load Story Dataset
with open("/content/Premchand - Godan2.1.txt", "r", encoding="utf-8") as file:
    raw_text = file.read().split("\n\n")  # Splitting stories by paragraphs

# ✅ Define Multiple Instructions to Add Variety
instructions = [
    "Write a Hindi short story about bravery.",
    "Write a Hindi short story about friendship.",
    "Write a Hindi short story about village life.",
    "Write a Hindi short story about struggle and success.",
    "Write a Hindi short story about kindness."
]

# ✅ Format Dataset for Ganga LLM
formatted_data = []
for story in raw_text:
    formatted_data.append({
        "instruction": random.choice(instructions),  # Randomly assign an instruction
        "input": "",  # Keep empty unless you want a structured prompt
        "output": story.strip()  # Story text as the response
    })

# ✅ Convert to Hugging Face Dataset format
dataset = Dataset.from_list(formatted_data)


In [ ]:
def tokenize_function(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"

    # Tokenize text
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # Manually shift the labels for loss computation
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# ✅ Apply Correct Tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/893 [00:00<?, ? examples/s]

In [ ]:
lora_config = LoraConfig(
    r=16,  # ✅ Increase rank for better fine-tuning
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,703,936 || all params: 997,066,752 || trainable%: 0.1709


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ganga-story-lora",
    per_device_train_batch_size=2,  # 🔽 Reduce if GPU memory is an issue
    gradient_accumulation_steps=4,
    learning_rate=2e-5,  # 🔽 Lowered LR for better stability
    num_train_epochs=7,  # 🔼 Enough epochs for convergence
    save_strategy="epoch",
    logging_strategy="epoch",  # ✅ Logs loss after each epoch
    evaluation_strategy="no",  # ✅ Evaluates after each epoch
    optim="adamw_torch",  # ✅ Better optimizer
    weight_decay=0.01,  # ✅ Helps prevent overfitting
    lr_scheduler_type="cosine",  # ✅ Smoothly reduces LR over time
    max_grad_norm=1.0,  # ✅ Prevents exploding gradients
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="none",  # ✅ Avoids logging to external trackers
    fp16=True,
)

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Now using structured dataset
)

# ✅ Train the Model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
111,7.427200
222,2.923100
333,2.604100
444,2.519100
555,2.484600
666,2.470200
777,2.443200


TrainOutput(global_step=777, training_loss=3.267370354101311, metrics={'train_runtime': 932.6998, 'train_samples_per_second': 6.702, 'train_steps_per_second': 0.833, 'total_flos': 1.7900800239992832e+16, 'train_loss': 3.267370354101311, 'epoch': 6.993288590604027})

In [ ]:
input_text = "### Instruction:\nWrite a Hindi short story.\n\n### Response:\n"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

output = model.generate(
    input_ids,
    max_length=300,
    do_sample=True,
    top_p=0.95,
    temperature=0.85,
    repetition_penalty=1.3  # 🔼 Higher to reduce word loops
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nGenerated Story:\n", generated_text)


Generated Story:
 ### Instruction:
Write a Hindi short story.

### Response:
पाठक का यह दायित्व बनता है कि वह अपने धर्म-शास्त्रों व मान्यताओं के अनुसार सत्य को जानता, समझ सकता है, और उसके अनुरूप आचरण करता है। पाठक या दर्शक इस बात पर अवश्य विचार करेंगे! इसी से लेखक की साख अच्छी बनती है! साहित्य समाज में मूल्यों, सिद्धांतों, परम्पराओं, विश्वासों तथा भाषा संबंधी विचारों से सम्बन्धित किसी भी विषय-विशेष अथवा मत-व्यवहार आदि से यदि लेखन का संबंध हो तो पाठक या श्रोता उस तक पहुँच सकेगा। इसलिए सभी विषयों/विचार-सामग्री हेतु एक ऐसा शब्द होना चाहिए जिसे सुनकर व्यक्ति, उसकी सोच तथा रचना-प्रक्रिया तक प्रभावित हो। उसका उद्देश्य तभी सार्थक होगा जब उसमें पाठकों और श्रोताओं दोनों के हित का ध्यान रखा जाएगा। इसलिए आज हमें यह कार्य बहुत आवश्यक लगता है; अतः हमने ऐसी रूपरेखा बनानी आरम्भ कर दी जिसके माध्यम से हम अधिक से अधिक लोगों तक पहुँचा सके। इससे हमारी लेखकीय क्षमता बढ़ने वाली है।इससे पहले केंद्रीय मंत्री स्मृति ईरानी ने कहा था कि कांग्रेस महासचिव प्रियंका वाड्रा किसानों की फसल बर्बाद होने पर आंसू बहा रही